# Threading vs Multiprocessing

We have two common approaches to run code in parallel (achieve multitasking and speed up your program): via threads or via multiple process.

## Process 

A process is an instance of a program, e.g. a Python interpreter. They are idepend from each other and do not share the same memory.

Key facts:
<ul>
    <li> A new process is started independently from the first process </li>
    <li> Takes advantage of multiple CPUs and cores </li>
    <li> Separate memory space </li>
    <li> Memory is not shared between processes </li>
    <li> One GIL (Global Interpreter lock) for each process, i.e. avoids GIL limitation </li>
    <li> Great for CPU-bound process </li>
    <li> Child processes are interruptable/killable </li>
    
    <li> Heavyweight </li>
    <li> Starting a process is slower that starting a thread </li>
    <li> Larger memory footprint </li>
    <li> IPC (interprocess communication) is more complicated </li>
</ul>

## Threads

A thread is an entity within a process that can be scheduled for execution (Also known as "leightweight process"). A Process can spawn multiple threads. The main difference is that all threads within a process share the same memory.

Key facts:
<ul>
    <li> Multiple threads can be spawned within one process </li>
    <li> Memory is shared between all threads </li>
    <li> Starting a thread is faster than starting a process </li>
    <li> Great for I/O-bound tasks </li>
    <li> Lightweight - low memory footprint </li>
    
    
    <li> One GIL for all threads, i.e. threads are limited by GIL </li>
    <li> Multithreading has no effect for CPU-bound tasks due to the GIL </li>
    <li> Not interruptible/killable -> be careful with memory leaks </li>
    <li> Increased the potention for race conditions </li>
    </ul>


### GIL: Global interpreter lock

This is a mutex (or a lock) that allows only one thread to hold control of the Python interpreter. This means that the GIL allows only one thread to execute at a time even in a multi-threaded architecture.
<ol>
    <li>A lock that allows only one thread at a time to execute in Python </li>
    <li>Needed in CPython because memory management is ont thread-safe </li>
    
    <li>Avoid: </li>
        <ol>
            <li>Use multiprocessing </li>
            <li>Use a different, free-threaded Python implementation (Jython, IronPython)</li>
            <li>Use Python as a wrapper for third party libraries(C/C++) -> numpy, scipy </li>
    </ol>           
</ol>

#### Why is it needed?

It is needed because CPython's (reference implementation of Python) memory management is not thread-safe. Python uses reference counting for memory management. It means that objects created in Python have a reference count variable that keeps track of the numbre of references that point to the object. When this count reaches zero, the memory occupied by the object is released. The problem was that this reference count variable needed protection from race conditions where two threads increase or decrease its value simultaneously. If this happens, it can cause either leaked memory that is never released or incorrectly release the memory while a reference to that object still exists.

#### How to avoid the GIL

The GIL is very controversial in the Python community. The main way to avoid the GIL is by using multiprocessing instead of threading. Another (however uncomfortable) solution would be to avoid the CPython implementation and use a free-threaded implementation like JPython or IronPython. A third option is to move parts of the application out into binary extensions modules, i.e. use Python as a wrapper for third party libraries (e.g. C/C++). This is thte parth taken by numpy and scipy.

## Threading in Python

Use the threading module.

Note: The following example usualy won't benefit from multiple threads since it is CPU-bound. It should just down the example of how to use threads.

In [ ]:
from threading import Thread

def square_numbers():
    for i in range(1000):
        result = i * i
        
if __name__ == "__main__":
    threads = []
    num_threads = 10
    
    # create threads and assign a function for each thread
    for i in range (num_threads):
        thread = Thread(target=square_numbers)
        threads.append(thread)
        
    # start all threads
    for thread in threads:
        thread.start()
        
    # wait for all threads to finish 
    # block the main thread until these threads are finished 
    for thread in threads:
        thread.join()

### When is Threading useful

Despite the GIL, it is useful for I/O-bound tasks when your program has to talk to slow devices, like a hard drive or a network connection. With threading the program can use the time waiting for these devices and intelligentl do other tasks in the meantime.

Example: Download website information from multiple sites. Use a thread for each site.`b

## Multiprocessing

Use the multiprocess module. The syntax is very similar to above.

In [ ]:
from multiprocessing import Process
import os

def square_numbers():
    for i in range (1000):
        result = i * i

if __name__ == "__main__":
    processes = []
    num_processes = os.cpu_count()
    
    # create processes and assign a function for each process
     for i in range(num_processes):
            process = Process(target=square_numbers)
            processes.append(process)
            
    # start all processes
    for process in processes:
        process.start()
        
    # wait for all processes to finish
    # block the main thread until these processes are finished
    for process in processes:
        process.join()

### When is Multiprocessing useful

It is useful for CPU-bound tasks that have to do a lot of CPU operations for a large amount of data and require a lot of computation time. With multiprocessing you can split the data into equal parts an do parallel computing on different CPUs.

Example: Calculate the square numbers for all numbers from 1 to 10000000. Divide the numbers into equal sized parts and use a process for each subset.